# Day 3 (t.test)  

I don't think I can run T-test on the data I started with [(univeristies) from the first two days](https://www.kaggle.com/poloniki/5-day-data-challenge-day-2), so I had to create new notebook.  

The task was to perform a T-test. So I had to choose two sets to compare and i choose to compare revenues of different museum types History and General Museums.

When I first I looked at data there was no sign of trouble.

In [ ]:
from scipy.stats import ttest_ind # just the t-test from scipy.stats
from scipy.stats import probplot # for a qqplot
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 
data = pd.read_csv('../input/museums.csv')
data.describe()

At that moment it get a little confusing. Do museums make that much? So there is a museum which made $5,840,349,000? Maybe I don't know smth.

Now I need to subset data for both types of museums.

In [ ]:
histmuseum = data['Museum Type'] == 'HISTORY MUSEUM'
genmuseum = data['Museum Type'] == 'GENERAL MUSEUM'
# hr = history museum revenue
hr = data[histmuseum]['Revenue']
# gr = general museum revenue
gr = data[genmuseum]['Revenue']

hr.head()

It seems like we have some duplicate and missing values and I remember I saw that unit on DataCamp. So will go there to look for solutions.

In [ ]:
# removing duplicates d=no dulicates
dhr = hr.drop_duplicates()
dgr = gr.drop_duplicates()
# removing NAN values n=no NaN
ndhr = dhr.dropna()
ndgr = dgr.dropna()

Lets see the distribution on the histogramm.

In [ ]:
fig = plt.figure(figsize=(10 , 4))
hist = ndgr
# Histogram
plt.hist(hist, bins=10, color='grey', edgecolor='white')
#Customizations
plt.xlabel('')
plt.ylabel('')
plt.title('')
plt.show()
plt.clf()

That simply made no sense to me. So I tried to normalize it by Z-score.

In [ ]:
z_score = (ndgr - ndgr.mean())/ndgr.std(ddof=0)
hist = z_score
# Histogram
plt.hist(hist, bins=10, color='grey', edgecolor='white')
#Customizations
plt.xlabel('')
plt.ylabel('')
plt.title('')
# Show and clear plot
plt.show()
plt.clf()


Well, that made no difference and it makes sense why.  Data is hugely disperse (2.189466e+08). I will have to work on it later to figure out hot to deal with it.

# Updated

I did! =) With the help of  [h4pZ](https://www.kaggle.com/h4pzzz) I solved the problem. **h4pZ** suggested to apply .log on each series. 

In [ ]:
# appling log to both series
logndhr = ndhr.apply(np.log)
logndgr = ndgr.apply(np.log)
# getting rid of all the inf
noinfndhr = logndhr > 0
noinfndgr = logndgr > 0
#finalized verison for both types
finHistory = logndhr[noinfndhr]
finGeneral = logndgr[noinfndgr]

Lets plot it and see what did we get.

In [ ]:
plt.figure(figsize=(14 , 7))
sns.distplot(finHistory)
sns.distplot(finGeneral)

Now that looks much better! Seems like two almost perfect normal distributions. Now we can get back to the task and perform a T-test. 
First I would like to see the QQ plot also.

In [ ]:
probplot(finHistory, dist="norm", plot=plt)
probplot(finGeneral, dist="norm", plot=plt)


Looks good. 
Now we need to define our hypotheses: 
* Our null hypothesis -  Hist sample have no significant difference with General sample
* Alt hypothesis - they are different.
alpa = 0.05

In [ ]:
ttest_ind(finHistory, finGeneral, equal_var=False)

So we got our P-value = almost 0 (**by the way when i was perfoming test without log - I got P-value=.7, which is bigger than our alpha of .5**).

Which means both samples are pretty much the same.

And to prove that point again I will plot two hists stole them from [here](https://www.kaggle.com/h4pzzz/5-day-data-challenge-pokemon-dataset-days-1-4) =)

Plots are not perfectly distributed, but I can say its pretty close.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Plotting the first histogram.
ax[0].hist(finHistory, edgecolor="white")
ax[0].axvline(finHistory.mean(), c="r")  # Plotting the mean value.
ax[0].set_title("Revenue distribution of History Museum's")

# Plotting the second histogram.
ax[1].hist(finGeneral, edgecolor="white")
ax[1].axvline(finGeneral.mean(), c="r")  # Plotting the mean value.
ax[1].set_title("Revenue distribution of General Museum's")